<a href="https://colab.research.google.com/github/kcat4lina/Spin-Win/blob/main/Atkins2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import random
from typing import List, Dict, Tuple

class SlotMachine:
    def __init__(self):
        self._reels = [
            ["Atkins", "Mayonnaise", "Ham", "Sausage", "Bacon", "Eggs", "Cheese", "Mayonnaise", "Sausage", "Butter", "Buffalo Wings", "Bacon", "Eggs", "Mayonnaise", "Steak", "Buffalo Wings", "Butter", "Cheese", "Eggs", "Atkins", "Bacon", "Mayonnaise", "Ham", "Cheese", "Eggs", "Scatter", "Butter", "Bacon", "Sausage", "Buffalo Wings", "Steak", "Butter"],
            ["Mayonnaise", "Buffalo Wings", "Steak", "Sausage", "Cheese", "Mayonnaise", "Ham", "Butter", "Bacon", "Steak", "Sausage", "Mayonnaise", "Ham", "Atkins", "Butter", "Eggs", "Cheese", "Bacon", "Sausage", "Buffalo Wings", "Scatter", "Mayonnaise", "Butter", "Cheese", "Bacon", "Eggs", "Buffalo Wings", "Mayonnaise", "Steak", "Ham", "Cheese", "Bacon"],
            ["Ham", "Butter", "Eggs", "Scatter", "Cheese", "Mayonnaise", "Butter", "Ham", "Sausage", "Bacon", "Steak", "Buffalo Wings", "Butter", "Mayonnaise", "Cheese", "Sausage", "Eggs", "Bacon", "Mayonnaise", "Buffalo Wings", "Ham", "Sausage", "Bacon", "Cheese", "Eggs", "Atkins", "Buffalo Wings", "Bacon", "Butter", "Cheese", "Mayonnaise", "Steak"],
            ["Ham", "Cheese", "Atkins", "Scatter", "Butter", "Bacon", "Cheese", "Sausage", "Steak", "Eggs", "Bacon", "Mayonnaise", "Sausage", "Cheese", "Butter", "Ham", "Mayonnaise", "Bacon", "Buffalo Wings", "Sausage", "Cheese", "Eggs", "Butter", "Buffalo Wings", "Bacon", "Mayonnaise", "Eggs", "Ham", "Sausage", "Steak", "Mayonnaise", "Bacon"],
            ["Bacon", "Scatter", "Steak", "Ham", "Cheese", "Sausage", "Butter", "Bacon", "Buffalo Wings", "Cheese", "Sausage", "Ham", "Butter", "Steak", "Mayonnaise", "Eggs", "Sausage", "Ham", "Atkins", "Butter", "Buffalo Wings", "Mayonnaise", "Eggs", "Ham", "Bacon", "Butter", "Steak", "Mayonnaise", "Sausage", "Eggs", "Cheese", "Buffalo Wings"]
        ]
        self._paytable = {
            "Atkins": {2: 5, 3: 50, 4: 500, 5: 5000},
            "Steak": {3: 40, 4: 200, 5: 1000},
            "Ham": {2: 2, 3: 30, 4: 150, 5: 500},
            "Buffalo Wings": {2: 2, 3: 25, 4: 100, 5: 300},
            "Sausage": {3: 20, 4: 75, 5: 200},
            "Eggs": {3: 20, 4: 75, 5: 200},
            "Butter": {3: 15, 4: 50, 5: 100},
            "Cheese": {3: 15, 4: 50, 5: 100},
            "Bacon": {3: 10, 4: 25, 5: 50},
            "Mayonnaise": {3: 10, 4: 25, 5: 50},
            "Scatter": {3: 5, 4: 25, 5: 100}
        }
        self._symbol_order = ["Atkins", "Steak", "Ham", "Buffalo Wings", "Sausage", "Eggs", "Butter", "Cheese", "Bacon", "Mayonnaise", "Scatter"]

    def _evaluate_line(self, line: List[str]) -> Tuple[str, int, int]:
        for symbol in self._symbol_order:
            count = 0
            for s in line:
                if s == symbol or (s == "Atkins" and symbol != "Scatter"):
                    count += 1
                else:
                    break
            if count >= 2 and symbol in self._paytable and count in self._paytable[symbol]:
                return symbol, count, self._paytable[symbol][count]
        return "", 0, 0

    def _count_scatters(self, matrix: List[List[str]]) -> int:
        return sum(symbol == "Scatter" for row in matrix for symbol in row)

    def calculate_probabilities(self) -> Dict:
        total_combinations = 32 ** 5
        combinations = {symbol: {2: 0, 3: 0, 4: 0, 5: 0} for symbol in self._symbol_order}
        scatter_combinations = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}

        for i1 in range(32):
            for i2 in range(32):
                for i3 in range(32):
                    for i4 in range(32):
                        for i5 in range(32):
                            line = [self._reels[0][i1], self._reels[1][i2], self._reels[2][i3], self._reels[3][i4], self._reels[4][i5]]
                            symbol, count, _ = self._evaluate_line(line)
                            if symbol and count >= 2:
                                combinations[symbol][count] += 1

                            scatter_count = sum(1 for s in line if s == "Scatter")
                            scatter_combinations[scatter_count] += 1

        probabilities = {
            symbol: {
                count: hits / total_combinations
                for count, hits in counts.items() if hits > 0
            }
            for symbol, counts in combinations.items()
        }

        scatter_probabilities = {
            count: hits / total_combinations
            for count, hits in scatter_combinations.items() if hits > 0
        }

        return probabilities, scatter_probabilities

    def calculate_rtp(self, probabilities: Dict, scatter_probabilities: Dict) -> float:
        rtp = 0
        for symbol, counts in probabilities.items():
            for count, prob in counts.items():
                if symbol in self._paytable and count in self._paytable[symbol]:
                    rtp += prob * self._paytable[symbol][count]

        for count, prob in scatter_probabilities.items():
            if count >= 3:
                rtp += prob * self._paytable["Scatter"][count]

        return rtp * 100  # Convert to percentage

def main():
    slot = SlotMachine()
    probabilities, scatter_probabilities = slot.calculate_probabilities()
    rtp = slot.calculate_rtp(probabilities, scatter_probabilities)

    print("Line Pay Combinations and Probabilities:")
    for symbol in slot._symbol_order:
        if symbol in probabilities:
            print(f"{symbol}:")
            for count, prob in probabilities[symbol].items():
                combinations = prob * (32 ** 5)
                print(f"  {count} in a row: {combinations:.0f} combinations, Probability: {prob:.8f}")

    print("\nScatter Probabilities:")
    for count, prob in scatter_probabilities.items():
        if count >= 3:
            combinations = prob * (32 ** 5)
            print(f"{count} Scatters: {combinations:.0f} combinations, Probability: {prob:.8f}")

    print(f"\nTotal RTP: {rtp:.2f}%")

if __name__ == "__main__":
    main()

Line Pay Combinations and Probabilities:
Atkins:
  2 in a row: 63488 combinations, Probability: 0.00189209
  3 in a row: 1984 combinations, Probability: 0.00005913
  4 in a row: 62 combinations, Probability: 0.00000185
  5 in a row: 2 combinations, Probability: 0.00000006
Steak:
  3 in a row: 38976 combinations, Probability: 0.00116158
  4 in a row: 3528 combinations, Probability: 0.00010514
  5 in a row: 504 combinations, Probability: 0.00001502
Ham:
  2 in a row: 401408 combinations, Probability: 0.01196289
  3 in a row: 50176 combinations, Probability: 0.00149536
  4 in a row: 6048 combinations, Probability: 0.00018024
  5 in a row: 1120 combinations, Probability: 0.00003338
Buffalo Wings:
  2 in a row: 516096 combinations, Probability: 0.01538086
  3 in a row: 66816 combinations, Probability: 0.00199127
  4 in a row: 6048 combinations, Probability: 0.00018024
  5 in a row: 864 combinations, Probability: 0.00002575
Sausage:
  3 in a row: 62208 combinations, Probability: 0.00185394
 